In this notebook the easily accessible revenues from google search are converted into USD and standardized in terms of units. The result is saved into revenues.csv

In [2]:
import pandas as pd
C610 = pd.read_excel('Data/2018/Investor Public Climate Change 2018 data_1804_extracted 17Apr19.xlsx', 
        sheet_name = 'C6.10',
        usecols = [1, 13, 14, 15, 19])

In [3]:
df = C610[:]
organizations = df['Organization'].unique()
df.dropna(inplace = True)

/home/jack/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
df.dropna(inplace = True)
for i, row in df.iterrows():
    if ('revenue' or 'sales') not in str(row[2]):
        df.drop([i], inplace = True)
df


/home/jack/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jack/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Organization,C6.10_C2_Describe your gross global combined Scope 1 and 2 emissions for the reporting year in metric tons CO2e per unit currency total revenue and provide any additional intensity metrics that are appropriate to your business operations. - Metric numerator (Gross global combined Scope 1 and 2 emissions),C6.10_C3_Describe your gross global combined Scope 1 and 2 emissions for the reporting year in metric tons CO2e per unit currency total revenue and provide any additional intensity metrics that are appropriate to your business operations. - Metric denominator,C6.10_C4_Describe your gross global combined Scope 1 and 2 emissions for the reporting year in metric tons CO2e per unit currency total revenue and provide any additional intensity metrics that are appropriate to your business operations. - Metric denominator: Unit total,C6.10_C8_Describe your gross global combined Scope 1 and 2 emissions for the reporting year in metric tons CO2e per unit currency total revenue and provide any additional intensity metrics that are appropriate to your business operations. - Reason for change
0,3i Group,750.76,unit total revenue,1.425000e+09,Intensity figure entered as zero here due to l...
2,3M Company,5840000.00,unit total revenue,3.165700e+04,Emission Reduction Activity
3,A2A,8179294.00,unit total revenue,5.796000e+09,No relevant changes due to the significant inc...
5,Aareal Bank AG,6114.00,unit total revenue,8.400000e+08,As the total revenue and the available referen...
6,ABB,1028000.00,unit total revenue,3.431200e+04,We performed well towards our 2020 target over...
...,...,...,...,...,...
3505,Zignago Vetro SpA,256485.11,unit total revenue,3.463720e+08,"The parameter has decreased relevantly (-10,68..."
3506,"Zimmer Biomet Holdings, Inc.",176136.00,unit total revenue,7.824100e+03,While Zimmer Biomet's Scope 1 and Scope 2 emis...
3507,ZORLU DOĞAL ELEKTRİK ÜRETİMİ A.Ş.,736444.00,unit total revenue,3.868810e+08,The revenue has increased 2.58% and our absolu...
3508,ZORLU ENERJİ ELEKTRİK ÜRETİM A.Ş.,220967.11,unit total revenue,2.508800e+08,The revenue has decreased 23.1% and our absolu...


In [6]:
df.to_csv('scope1+2.csv')

In [5]:
companies = set(organizations) - set(df['Organization'])
len(companies)

411

Setting up the selenium headless driver on Firefox and performing automated google search for the companies' revenues

In [20]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, re

DRIVER = './geckodriver'
browser_locale = 'en-GB'

ffox_options = webdriver.FirefoxOptions()
ffox_options.add_argument('-headless')

ffox_profile = webdriver.FirefoxProfile()
ffox_profile.set_preference('intl.accept_languages', browser_locale)
browser = webdriver.Firefox(executable_path=DRIVER, options=ffox_options, firefox_profile=ffox_profile)

try:
    for company in list(companies)[390:]:

        browser.get("http://www.google.com")
        time.sleep(2)
        que=browser.find_element_by_xpath("//input[@name='q']")
        que.send_keys('revenue ' + company)
        que.send_keys(Keys.RETURN)
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        
        div_search = soup.find('div',{'id':'search'}).text.replace(u'\xa0', u' ')
        if 'revenue' in div_search.lower():
            revenues[company] = re.search(r'[0-9]{1,3}\.[0-9]{1,3}\s[mb]illion\s[A-Z]{3}', div_search)
            
finally:
    browser.quit()

In [28]:
non0revenues = {}
for k,v in revenues.items():
    if v:
        non0revenues[k] = v.group()
non0revenues

{'Bharat Forge': <re.Match object; span=(34, 51), match='74.85 billion INR'>,
 'Marvell Technology Group, Ltd.': <re.Match object; span=(46, 63), match='2.866 billion USD'>,
 'SAS': <re.Match object; span=(35, 51), match='3.27 billion USD'>,
 'Crest Nicholson PLC': <re.Match object; span=(37, 54), match='1.086 billion GBP'>,
 'Laurentian Bank of Canada': <re.Match object; span=(47, 64), match='915.5 million USD'>,
 'BillerudKorsnäs': <re.Match object; span=(37, 53), match='23.7 billion SEK'>,
 'The Southern Company': <re.Match object; span=(38, 54), match='23.4 billion USD'>,
 'Quebecor Inc.': <re.Match object; span=(238, 255), match='4.122 billion CAD'>,
 'Braskem S/A': <re.Match object; span=(29, 45), match='14.7 billion USD'>,
 'JSW Steel': <re.Match object; span=(35, 52), match='847.6 billion INR'>,
 'Capital Power Corporation': <re.Match object; span=(35, 52), match='1.251 billion CAD'>,
 'Costco Wholesale Corporation': <re.Match object; span=(28, 45), match='152.7 billion USD'>,


In [31]:
revenues = {}
for k,v in non0revenues.items():
    if v:
        revenues[k] = v.split(' ')
revenues

{'Bharat Forge': ['74.85', 'billion', 'INR'],
 'Marvell Technology Group, Ltd.': ['2.866', 'billion', 'USD'],
 'SAS': ['3.27', 'billion', 'USD'],
 'Crest Nicholson PLC': ['1.086', 'billion', 'GBP'],
 'Laurentian Bank of Canada': ['915.5', 'million', 'USD'],
 'BillerudKorsnäs': ['23.7', 'billion', 'SEK'],
 'The Southern Company': ['23.4', 'billion', 'USD'],
 'Quebecor Inc.': ['4.122', 'billion', 'CAD'],
 'Braskem S/A': ['14.7', 'billion', 'USD'],
 'JSW Steel': ['847.6', 'billion', 'INR'],
 'Capital Power Corporation': ['1.251', 'billion', 'CAD'],
 'Costco Wholesale Corporation': ['152.7', 'billion', 'USD'],
 'Lancashire Holdings': ['442.6', 'million', 'USD'],
 'The Spar Group Ltd': ['34.5', 'billion', 'EUR'],
 'Teradyne Inc.': ['2.1', 'billion', 'USD'],
 'Cranswick': ['1.464', 'billion', 'GBP'],
 'Almirall S.A.': ['859.3', 'million', 'EUR'],
 'Banco Bradesco S/A': ['42.7', 'billion', 'USD'],
 'AT&T Inc.': ['170.8', 'billion', 'USD'],
 'The AES Corporation': ['10.74', 'billion', 'USD'],


In [33]:
for k,v in revenues.items():
    if v[1] == 'billion':
        revenues[k] = (float(v[0])*10e9, v[2])
    if v[1] == 'million':
        revenues[k] = (float(v[0])*10e6, v[2])
revenues

{'Bharat Forge': (748500000000.0, 'INR'),
 'Marvell Technology Group, Ltd.': (28660000000.0, 'USD'),
 'SAS': (32700000000.0, 'USD'),
 'Crest Nicholson PLC': (10860000000.0, 'GBP'),
 'Laurentian Bank of Canada': (9155000000.0, 'USD'),
 'BillerudKorsnäs': (237000000000.0, 'SEK'),
 'The Southern Company': (234000000000.0, 'USD'),
 'Quebecor Inc.': (41220000000.0, 'CAD'),
 'Braskem S/A': (147000000000.0, 'USD'),
 'JSW Steel': (8476000000000.0, 'INR'),
 'Capital Power Corporation': (12509999999.999998, 'CAD'),
 'Costco Wholesale Corporation': (1527000000000.0, 'USD'),
 'Lancashire Holdings': (4426000000.0, 'USD'),
 'The Spar Group Ltd': (345000000000.0, 'EUR'),
 'Teradyne Inc.': (21000000000.0, 'USD'),
 'Cranswick': (14640000000.0, 'GBP'),
 'Almirall S.A.': (8593000000.0, 'EUR'),
 'Banco Bradesco S/A': (427000000000.0, 'USD'),
 'AT&T Inc.': (1708000000000.0, 'USD'),
 'The AES Corporation': (107400000000.0, 'USD'),
 'Ipsos': (17500000000.0, 'EUR'),
 'Cia Energética de São Paulo S.A. - CESP':

In [34]:
for k,v in revenues.items():
    if v[1] == 'INR':
        revenues[k] = v[0] * 0.013
    elif v[1] == 'GBP':
        revenues[k] = v[0] * 1.25
    elif v[1] == 'SEK':
        revenues[k] = v[0] * 0.1
    elif v[1] == 'NZD':
        revenues[k] = v[0] * 0.61
    elif v[1] == 'CHF':
        revenues[k] = v[0] * 1.03
    elif v[1] == 'CAD':
        revenues[k] = v[0] * 0.72
    elif v[1] == 'CBD':
        revenues[k] = v[0] * 0.189154
    elif v[1] == 'JPY':
        revenues[k] = v[0] * 0.0094
    elif v[1] == 'NOK':
        revenues[k] = v[0] * 0.094
    elif v[1] == 'HKD':
        revenues[k] = v[0] * 0.13
    elif v[1] == 'DKK':
        revenues[k] = v[0] * 0.15
    elif v[1] == 'AUD':
        revenues[k] = v[0] * 0.65
    elif v[1] == 'KRW':
        revenues[k] = v[0] * 0.00082
    elif v[1] == 'CNY':
        revenues[k] = v[0] * 0.14
    elif v[1] == 'SGD':
        revenues[k] = v[0] * 0.71
    elif v[1] == 'EUR':
        revenues[k] = v[0] * 1.09
    else:
        revenues[k] = v[0]
        
revenues

{'Bharat Forge': 9730500000.0,
 'Marvell Technology Group, Ltd.': 28660000000.0,
 'SAS': 32700000000.0,
 'Crest Nicholson PLC': 13575000000.0,
 'Laurentian Bank of Canada': 9155000000.0,
 'BillerudKorsnäs': 23700000000.0,
 'The Southern Company': 234000000000.0,
 'Quebecor Inc.': 29678400000.0,
 'Braskem S/A': 147000000000.0,
 'JSW Steel': 110188000000.0,
 'Capital Power Corporation': 9007199999.999998,
 'Costco Wholesale Corporation': 1527000000000.0,
 'Lancashire Holdings': 4426000000.0,
 'The Spar Group Ltd': 376050000000.0,
 'Teradyne Inc.': 21000000000.0,
 'Cranswick': 18300000000.0,
 'Almirall S.A.': 9366370000.0,
 'Banco Bradesco S/A': 427000000000.0,
 'AT&T Inc.': 1708000000000.0,
 'The AES Corporation': 107400000000.0,
 'Ipsos': 19075000000.0,
 'Cia Energética de São Paulo S.A. - CESP': 4224000000.0,
 'B2W Companhia Global do Varejo': 16000000000.0,
 'Welltower Inc.': 37750000000.0,
 'DNO International ASA': 3474000000.0,
 'IHI Corporation': 225000000000.0,
 'Agropur Cooperati

In [36]:
revenuesUSD = {}
for k,v in revenues.items():
    revenuesUSD[k] = int(v)
revenuesUSD = pd.DataFrame(revenuesUSD.items())

In [39]:
revenuesUSD.to_csv('revenues.csv')